## The imports

In [1]:
import zarr
import xarray as xr
import time
import dask
from intake import open_catalog


# The workers

In [2]:
from dask.distributed import Client
from dask_gateway import Gateway

gateway = Gateway()
cluster = gateway.new_cluster()
options = gateway.cluster_options()
#options.worker_cores = 2
#options.worker_memory = 8


In [3]:
c = Client(cluster)
c

Client Scheduler: gateway://traefik-gcp-uscentral1b-prod-dask-gateway.prod:80/prod.57d8c79623864935b6b8a9479d316fc9 Dashboard: /services/dask-gateway/clusters/prod.57d8c79623864935b6b8a9479d316fc9/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [5]:
from dask.utils import ensure_dict, format_bytes
workers = c.scheduler_info()["workers"]
text="Workers= " + str(len(workers))
memory = [w["memory_limit"] for w in workers.values()]
if all(memory):
    text += ", Memory=" + format_bytes(sum(memory))
print(text)


KeyError: 'workers'

# The data

In [ ]:
%%time 

import xarray as xr
import gcsfs

fs = gcsfs.GCSFileSystem('pangeo-181919', requester_pays=True)

zmap = fs.get_mapper('pangeo-meom/autre-eNATL60-BLBT02-SSH-1h/eNATL60-BLBT02-SSH-1h')
ds = xr.open_zarr(zmap)


In [ ]:
ds

In [ ]:
%time mean=ds.sossheig.mean(dim='time_counter')

In [ ]:
mean

In [ ]:
%time mean.load()

In [ ]:
%time mean_space=ds.sossheig.mean(dim={'x','y'})

In [ ]:
mean_space

In [ ]:
%time mean_space.load()